# Modification of the dataset data.lmdb

## Voronoi

In [ ]:
from ocpmodels.datasets import SinglePointLmdbDataset

import numpy as np
from mendeleev import element
from scipy.spatial import distance_matrix
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
import pandas as pd
import copy
import torch
import lmdb
import re

def getAtomSequence (sequence) :
    result = list([[sequence[0], 1]])
    for i in range(1, len(sequence)) :
        if sequence[i] == result[-1][0] :
            result[-1][1] += 1
        else :
            result.append([sequence[i], 1])
    return dict(result)

def structureToVASP(structure, file='POSCAR', str_name='structure', relaxed=False) :
    with open(file + ('_relaxed' if relaxed else ''), 'w') as f :
        f.write(str_name + '\n')
        f.write(str(1.0) + '\n')
        for axis in np.array(structure.cell[0]) :
            for i in range(3) :
                f.write(str(axis[i]) + '   ')
                if i == 2 :
                    f.write('\n')
        atoms = getAtomSequence(np.array(structure.atomic_numbers, dtype=int))
        for k in atoms.keys() :
            f.write('   ' + element(round(k)).symbol)
        f.write('\n')
        for v in atoms.values() :
            f.write('   ' + str(round(v)))
        f.write('\n')        
        f.write('Cartesian\n')
        for position in np.array(structure.pos if not relaxed else structure.pos_relaxed)   :
            for i in range(3) :
                f.write(str(position[i]) + '   ')
                if i == 2 :
                    f.write('\n')            
    return None

def getTranslations (positions, cell) :
    result = np.array(positions)
    #print(result.shape)
    for i in range(-1,2) :
        for j in range(-1,2) :
            for k in range(-1,2) :
                if (i == 0) and (j == 0) and (k == 0) :
                    continue
                result = np.vstack((result, np.array(positions) + 
                                    i*np.array(cell[0][0]) + 
                                    j*np.array(cell[0][1]) +
                                    k*np.array(cell[0][2])) )
    return result

def getOffsets (positions) :
    result = np.zeros_like(positions, dtype=int)
    #print(result.shape)
    for i in range(-1,2) :
        for j in range(-1,2) :
            for k in range(-1,2) :
                if (i == 0) and (j == 0) and (k == 0) :
                    continue
                result = np.vstack((result, np.zeros_like(positions, dtype=int) + [i,j,k]) )
    return result

import functions, polyhedron, graph, geometry
from scipy.spatial import Voronoi
from polyhedron import Polyhedron
import geometry as gm
import numpy as np

 
class Voro:

    def __init__(self, points, central_ps, labels):
        """
        :param points: list of points coordinates, [[float, float, float], ...]
        :param central_ps: indexes of centrals points, list of int
        :param labels: list of labels
        """
        self.points = np.array(points)
        self.central_ps = central_ps
        self.labels = labels
        self.vor = Voronoi(points)
        self.p_adjacency = self.calc_p_adjacency()
        self.polyhedrons = self.construct_polyhedrons()
        self.rsds = np.array([(3 * p.volume / (4 * np.pi)) ** (1 / 3.) for p in self.polyhedrons])
        self.angles = self.calc_angles()
        self.direct_neighbors = self.find_direct_neighbors()

    def calc_p_adjacency(self):
        """
        Calculation of points adjacency list (points are adjacent if the domains of the points are adjacent)
        :return: self.p_adjacency: points adjacency list, [[int, ...], ...]
        """
        p_adjacency = [[] for _ in range(len(self.points))]
        for p1, p2 in self.vor.ridge_dict.keys():
            p_adjacency[p1] += [p2]
            p_adjacency[p2] += [p1]
        self.p_adjacency = p_adjacency
        return self.p_adjacency

    def construct_polyhedrons(self):
        """
        Construct of the polyhedrons
        :return: self.polyhedrons, Voronoi polyhedra, list of Polyhedron objects
        """
        self.polyhedrons = []
        for i in self.central_ps:
            faces = []
            region = self.vor.regions[self.vor.point_region[i]]
            new_ind = {o_i: n_i for n_i, o_i in enumerate(region)}
            if -1 in region:
                raise RuntimeError("The domain for \"" + str(i) + "\" point is not closed!")
            for j in self.p_adjacency[i]:
                common_vs = self.vor.ridge_dict.get((i, j))
                if common_vs is None:
                    common_vs = self.vor.ridge_dict[(j, i)][::-1]
                if i != j and common_vs is not None and len(common_vs) > 2:
                    faces += [[new_ind[o_i] for o_i in common_vs]]
                else:
                    raise RuntimeError("The Voronoi decomposition is failed!")
            faces = np.array([np.array(f) for f in faces])
            self.polyhedrons += [Polyhedron(self.vor.vertices[region], region, faces, find_order=False)]
        return self.polyhedrons

    def calc_angles(self):
        """
        Calculation of solid angles
        :return: self.angles, solid angles between adjacent Voronoi polyhedra, {(int, int): float, ...}
        """
        self.angles = {}
        for i, i1 in enumerate(self.central_ps):
            angles = []
            cp = self.points[i1]
            for j, i2 in enumerate(self.p_adjacency[i1]):
                angles += [sum([abs(gm.calc_solid_angle(cp, s)) for s in self.polyhedrons[i].faces[j].simplexes])]
            angles = 100 * np.array(angles) / sum(angles)
            for j, i2 in enumerate(self.p_adjacency[i1]):
                self.angles[(i1, i2)] = angles[j]
                self.angles[(i2, i1)] = angles[j]
        return self.angles

    def find_direct_neighbors(self):
        """
        Find the direct neighbors
        :return: self.direct_neighbors: direct Voronoi polyhedra neighbors, {(int, int): bool}
        """
        self.direct_neighbors = {}
        for i, i1 in enumerate(self.central_ps):
            p1 = self.points[i1]
            for j, i2 in enumerate(self.p_adjacency[i1]):
                p2 = self.points[i2]
                if self.polyhedrons[i].faces[j].is_inside(gm.calc_centroid([p1, p2])):
                    self.direct_neighbors[(i1, i2)] = True
                    self.direct_neighbors[(i2, i1)] = True
        return self.direct_neighbors

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import pickle

## New properties 1

In [ ]:
def newProperties(data) :
    keys = []
    result = []
    
    points = getTranslations(data.pos, data.cell)
    atom_index = np.array(list(range(data.natoms)) * round(len(points) / len(data.pos)))
    offsets = getOffsets(data.pos)
    voro = Voro(points, range(data.natoms), list(range(len(points))))
    
    volumes = np.array(list(map(lambda x: x.volume, voro.polyhedrons)))
    surface_areas = np.array(list(map(lambda x: x.area, voro.polyhedrons)))
    rsds = voro.rsds
    
    keys.extend(['voronoi_volumes', 'voronoi_surface_areas',
                'spherical_domain_radii'])
    result.extend([volumes, surface_areas, rsds])    
    
    df = pd.DataFrame(voro.angles.keys(), columns=['VA_p1', 'VA_p2'])
    #print(df)
    df['cell_offsets'] = list(map(tuple, offsets[df['VA_p1']] - offsets[df['VA_p2']])) 
    #sign correspond to original data
    df['VA_p1_corr'] = atom_index[df['VA_p1']]
    df['VA_p2_corr'] = atom_index[df['VA_p2']]
    df['direct_neighbor'] = list(map(lambda x: 1 if x in voro.direct_neighbors.keys() else 0, voro.angles.keys()))
    k = np.array(list(voro.angles.keys()))
    df['distance'] = np.array([euclidean(points[pair[0]], points[pair[1]]) for pair in k])
    df['solid_angle'] = voro.angles.values()
    df['to_keep'] = ~df.duplicated(subset=['VA_p1_corr', 'VA_p2_corr', 'cell_offsets'], keep='first')
    df = df[df['to_keep']].drop(labels=['to_keep', 'VA_p1', 'VA_p2'], axis=1).reset_index(drop=True)
    
    keys.extend(['cell_offsets_new', 'distances_new', 'contact_solid_angles',
                'direct_neighbor', 'edge_index_new'])
    result.extend([np.array(list(map(np.array, df['cell_offsets'].values))), df['distance'].values, 
                   df['solid_angle'].values, df['direct_neighbor'].values, 
                   df[['VA_p1_corr', 'VA_p2_corr']].values.T])  
    
    return dict(zip(keys, result))

def processDataset (dataset_path, dataset_batch_size=5004) :
    dataset_path_modified = re.sub('\.lmdb', '_mod.lmdb', dataset_path)
    print('###########################################')
    print(dataset_path_modified)
    print()
    
    dataset = SinglePointLmdbDataset({"src": dataset_path})
    print('Original dataset size is {}'.format(len(dataset)))
    print()
    
    dataset_target = lmdb.open(
        dataset_path_modified,
        map_size=int(1e9*50), #~ 50 Gbyte
        subdir=False,
        meminit=False,
        map_async=True,)
    
    print('Batch info:')
    
    for batch in range(len(dataset) // dataset_batch_size + 
                       (1 if len(dataset) % dataset_batch_size != 0 else 0)) :
        print('{}: from {} to {}'.format(batch, batch * dataset_batch_size, 
                                         min(len(dataset), (batch+1) * dataset_batch_size) - 1))
    
    print()
    for batch in range(len(dataset) // dataset_batch_size + 
                       (1 if len(dataset) % dataset_batch_size != 0 else 0)) :
        
        dataset_under_process = [dataset[i] for i in 
                                 range(batch * dataset_batch_size, 
                                       min(len(dataset), (batch+1) * dataset_batch_size))]
        print('Structures from {} to {} are under process...'.format(batch * dataset_batch_size, 
                                                batch * dataset_batch_size + len(dataset_under_process) - 1))
        
        res = Parallel(n_jobs=-1)(delayed(newProperties)(dataset_under_process[i]) 
                                  for i in tqdm(range(len(dataset_under_process))))
        #res = [dict() for i in range(len(dataset_under_process))]
        print('are stored to a file...')
        
        for structure_id in tqdm(range(len(dataset_under_process))) :
            txn = dataset_target.begin(write=True)
            data = dataset_under_process[structure_id]
            
            for new_data_key in res[0].keys() :
                #print(new_data_key)
                data[new_data_key] = torch.from_numpy(res[structure_id][new_data_key])
                #print(data[new_data_key])
            txn.put(f"{structure_id + batch * dataset_batch_size}".encode("ascii"), 
                    pickle.dumps(data, protocol=-1))
            txn.commit()
            dataset_target.sync()
    
    dataset_target.close()
    return None

## New properties 2

In [ ]:
def newProperties2(data) :
    ei = data['edge_index_new'].numpy()
    di = data['distances_new'].numpy()
    dn = data['direct_neighbor'].numpy()
    co = data['cell_offsets_new'].numpy()
    co = list(map(tuple, co))

    # тут можно добавить фильтры на связность (по ближайшим соседям или по максимальным длинам)

    Nedges = di.shape[0]
    #print(dict(enumerate(co)),)
    res_even = [calcAnglesForEdge(edgeN, dict(enumerate(ei.T)), dict(enumerate(di)), dict(enumerate(co)), 
                                  data['pos'].numpy(), data['cell'].numpy()) for edgeN in range(Nedges)[::2]]
    # accounting for inverse edges
    res_odd = copy.deepcopy(res_even)
    for i in range(len(res_odd)) :
        res_odd[i]['edge_theta'] *= -1 
        res_odd[i]['edge_theta'] += np.pi
        res_odd[i]['edge_phi'] *= -1

    res = []
    for i in range(len(res_odd)) :
        res += [res_even[i]] + [res_odd[i]]
    return {'edge_angles' : res}
        

def calcAnglesForEdge(edgeN, edge_by_edgeN, distance_by_edgeN, cell_offset_by_edgeN, positions, cell) :
    edge = edge_by_edgeN[edgeN]
    edge_positions = np.vstack([positions[edge[i]] for i in range(2)])
    #print('EP', edge_positions)
    #print(edgeN, len(edge_by_edgeN), len(distance_by_edgeN), len(cell_offset_by_edgeN), positions, cell)
    edge_positions[1] -= np.matmul(cell_offset_by_edgeN[edgeN], cell)[0] # update the second position with offset
    centroid = np.mean(edge_positions, axis=0)
    #print('centroid', centroid)
    v0 = edge_positions[1] - edge_positions[0] #edge vector
    #print(v0)
    v0 = v0/np.linalg.norm(v0) #edge unit direction
    #searching for the neighbors of the edge
    edge_neighborsN = [p for p in edge_by_edgeN.keys() if any([edge_by_edgeN[p][0] == edge[i] 
                                                               for i in range(2 if edge[0] != edge[1] else 1)]) ]
    #print(edge_neighborsN)
    edge_neighborsN.remove(edgeN) #remove edge itself
    edge_neighborsN.remove(edgeN + 1) #remove edge invert to the given
    #print(edge_neighborsN)
    df = pd.DataFrame([[*edge_by_edgeN[i], distance_by_edgeN[i], cell_offset_by_edgeN[i]] 
                          for i in edge_neighborsN], columns=['start', 'end', 'distance','cell_offset'])
    df['edge_id'] = edge_neighborsN
    #df['to_keep'] = ~df.duplicated(subset=['end', 'cell_offset'])
    #df = df[df['to_keep']].drop('to_keep', axis=1)
   
    result_theta = []
    result_angle_to_z = []
    plane_proj = [] #to evaluate phi of the neighboring edges further
    #centroid2neighbor_dist = []
    for i in range(2 if edge[0] != edge[1] else 1): # loop over the nodes of edge
        centroid2node_vec = edge_positions[i] - centroid
        for NES, NEE, offset in df[(df['start'] == edge[i])][['start', 'end', 'cell_offset']].values:
            neighbor_edge = (NES, NEE)
            neighbor_edge_positions = np.vstack([positions[neighbor_edge[i]] for i in range(2)])
            neighbor_edge_positions[1] -= np.matmul(np.array(offset), cell)[0] 
            node2neighbor_vec = neighbor_edge_positions[1] - neighbor_edge_positions[0]
            v1 = node2neighbor_vec + centroid2node_vec #centroid2neighbor_vec
            #centroid2neighbor_dist += [np.linalg.norm(v1)]
            #print(v1)
            #print(v1, v0, np.dot(v0, v1))
            plane_proj += [(v1 - v0 * np.dot(v0, v1))] #centroid2neighbor_vec proj on edge normal plane
            v1 = v1/np.linalg.norm(v1) #centroid2neighbor unit direction
            #print(v1)
            result_theta += [np.arccos(np.clip(np.dot(v0, v1), -1, 1))]
            result_angle_to_z += [np.arccos(np.clip(np.dot(np.array([0,0,1]), v1), -1, 1))]
        #print('loop is over')
    #print(edgeN, df.shape, len(result_theta), len(result_angle_to_z))
    #df['cent2neigh_dist'] = centroid2neighbor_dist
    df['edge_theta'] = result_theta
    df['edge_to_z'] = result_angle_to_z
    #print(np.argmin(df['distance']))
    v1 = plane_proj[np.argmin(np.abs(df['edge_theta'] - np.pi/2))] # proj of centroid2most_incline neighbor 
    #if np.abs(np.linalg.norm(v1)) < 1e-7 :
    #    print(v1, np.linalg.norm(v1), np.argmin(df['cent2neigh_dist']))
    #    print(df)
    v1 = v1/np.linalg.norm(v1)                  # unit direction
    v2 = np.cross(v0, v1)                       # third axis
    Rm = np.vstack([v1, v2, v0]).T              # rotation matrix
    # rotation matrix transforms standard xyz CS to rotated z' -> edge direction, x' -> nearest neighbor
    #print(Rm)
    invRm = np.linalg.inv(Rm)                   # inverse rotation matrix 
    #print(invRm)
    plane_proj_rot = np.array(list(map(lambda x: np.matmul(invRm, np.array(x).reshape(-1,1)).reshape(1,-1)[0], 
                                  plane_proj)))
    #print(plane_proj_rot)
    df['edge_phi'] = list(map(lambda x: np.angle(complex(x[0], x[1])), plane_proj_rot))
    
    return df[['edge_id', 'edge_theta', 'edge_to_z', 'edge_phi']].set_index('edge_id')

def processDataset2 (dataset_path, dataset_batch_size=5004) :
    dataset_path_modified = re.sub('_mod\.lmdb', '_mod2.lmdb', dataset_path)
    print('###########################################')
    print(dataset_path_modified)
    print()
    
    dataset = SinglePointLmdbDataset({"src": dataset_path})
    #dataset = [dataset[i] for i in range(0, 12)]
    print('Original dataset size is {}'.format(len(dataset)))
    print()
    
    dataset_target = lmdb.open(
        dataset_path_modified,
        map_size=int(1e9*50), #~ 50 Gbyte
        subdir=False,
        meminit=False,
        map_async=True,)
    
    print('Batch info:')
    
    for batch in range(len(dataset) // dataset_batch_size + 
                       (1 if len(dataset) % dataset_batch_size != 0 else 0)) :
        print('{}: from {} to {}'.format(batch, batch * dataset_batch_size, 
                                         min(len(dataset), (batch+1) * dataset_batch_size) - 1))
    
    print()
    for batch in range(len(dataset) // dataset_batch_size + 
                       (1 if len(dataset) % dataset_batch_size != 0 else 0)) :
        
        dataset_under_process = [dataset[i] for i in 
                                 range(batch * dataset_batch_size, 
                                       min(len(dataset), (batch+1) * dataset_batch_size))]
        print('Structures from {} to {} are under process...'.format(batch * dataset_batch_size, 
                                                batch * dataset_batch_size + len(dataset_under_process) - 1))
        
        
        
        res = Parallel(n_jobs=-1)(delayed(newProperties2)(dataset_under_process[i]) 
                                  for i in tqdm(range(len(dataset_under_process))))
        #res = [dict() for i in range(len(dataset_under_process))]
        print('are stored to a file...')
        
        for structure_id in tqdm(range(len(dataset_under_process))) :
            txn = dataset_target.begin(write=True)
            data = dataset_under_process[structure_id]
            
            for new_data_key in res[0].keys() :
                #print(new_data_key)
                data[new_data_key] = res[structure_id][new_data_key]
                #print(data[new_data_key])
            txn.put(f"{structure_id + batch * dataset_batch_size}".encode("ascii"), 
                    pickle.dumps(data, protocol=-1))
            txn.commit()
            dataset_target.sync()
    
    dataset_target.close()
    return None

## Proccesing

### Process data.lmdb to data_mod.lmdb

**Added feautures.** </br>
atom: \['voronoi_volumes', 'voronoi_surface_areas', 'spherical_domain_radii'], </br>
bond; \['cell_offsets_new', 'distances_new', 'contact_solid_angles', 'direct_neighbor', 'edge_index_new']

In [ ]:
#processDataset('/Users/Eremin/OneDrive/Share/10k/train/data.lmdb')
#processDataset('/Users/Eremin/OneDrive/Share/all/val_ood_both/data.lmdb')
#processDataset('/Users/Eremin/OneDrive/Share/all/test_ood_both/data.lmdb')

### Process data_mod.lmdb to data_mod2.lmdb

**Added feautures.** </br>
angles: \['edge_id', 'edge_theta', 'edge_to_z', 'edge_phi'], </br>

In [ ]:
dataset_path = '../../../ocp_datasets/data/is2re/all/val_ood_both/'

In [ ]:
processDataset2(dataset_path + 'data_mod.lmdb' )

**Test**

In [ ]:
data_ori = SinglePointLmdbDataset({"src": dataset_path + 'data_mod2.lmdb'})
data_ori[0].keys

In [ ]:
data_ori[0]['edge_angles'][0]